In [1]:
import ROOT as r
import math
import array
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, '../include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE
from UTILITIES import *
from SIMFITS2D import DistributionFits2D
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend


Welcome to JupyROOT 6.26/06


In [2]:
#block id

In [3]:
pas="1"
config="3"

In [4]:
W2min=CONFIG.Function_JSON("W2min",f"../config/cuts{config}.cfg")
W2max=CONFIG.Function_JSON("W2max",f"../config/cuts{config}.cfg")
dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
dymin=CONFIG.Function_JSON("dymin",f"../config/cuts{config}.cfg")
dymax=CONFIG.Function_JSON("dymax",f"../config/cuts{config}.cfg")
dybgmin=CONFIG.Function_JSON("dybgmin",f"../config/cuts{config}.cfg")
dybgmax=CONFIG.Function_JSON("dybgmax",f"../config/cuts{config}.cfg")
coinmin=CONFIG.Function_JSON("coinmin",f"../config/cuts{config}.cfg")
coinmax=CONFIG.Function_JSON("coinmax",f"../config/cuts{config}.cfg")
#dymin=-.984
#dymax=.9
#W2max=1.75
#dybgmin=-1.2
#dybgmax=1.2
print(f'W2min: {W2min}')
print(f'W2max: {W2max}')
print(f'dxmin: {dxmin}')
print(f'dxmax: {dxmax}')
print(f'dymin: {dymin}')
print(f'dymax: {dymax}')
print(f'dybgmin: {dybgmin}')
print(f'dybgmax: {dybgmax}')
print(f'Coin Min: {coinmin} Coin Max: {coinmax}')

W2min: -0.5
W2max: 1.7
dxmin: -0.4
dxmax: 0.4
dymin: -0.5
dymax: 0.5
dybgmin: -1.3
dybgmax: 1.2
Coin Min: 88 Coin Max: 100


In [5]:
#bb.tr.n==1&&bb.ps.e>0.2&&abs(bb.tr.vz[0])<0.28&&bb.gem.track.nhits>3
#bb.tr.n==1&&bb.ps.e>0.2&&abs(bb.tr.vz[0])<0.28&&bb.gem.track.nhits>3&&abs(bb.etot_over_p-0.92)<0.2
#&&sbs.hcal.e>0.01&&bb.ps.e+bb.sh.e>1.7

In [6]:
import ROOT as r

import ROOT as r

def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
rootfile_path = "../outfiles/HCal_data_GEN3_sbs100p_nucleon_np_model1.root"
list_branches(rootfile_path)


Branches in tree 'Tout':
runnum
WCut
pCut
nCut
fiduCut
coinCut
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
xtgt
ytgt
thtgt
phtgt
thetabend
xfp
yfp
thfp
phfp
trP
trPx
trPy
trPz
ePS
xPS
eSH
xSH
ySH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
ngrinch_hits
xGRINCH
yGRINCH
coin_time
hcal_time
bbcal_time
nhodo_clus
hodo_time
BPMAx
BPMAy
Rasterx
Rastery
Raster2x
Raster2y
helicity
IHWP
hcalE
hcalX
hcalY
pblkid
tdc
atime
hodotmean
nblk
mag
run
tar
cblkid
cblkatime
cblktime
cblke
nclus
cid
cx
cy
catime


Error in <TList::Clear>: A list is accessing an object (0x9fc38c0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5f7bd60) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x5f7c190) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa051ba0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa051f10) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa052280) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa0527b0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa052c60) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa062e50) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa076bc0) alread

In [7]:


def Load_HCal(config):
    rootfile = f"../outfiles/HCal_data_GEN3_sbs100p_nucleon_np_model1.root"
    rootfile = f"../outfiles/oldhcal.root"
    
    C = r.TChain("Tout")
    C.Add(rootfile)
    C.SetBranchStatus("*", 0)
   
    
    #HCAL VARS_____________________________________
    C.SetBranchStatus("cx", 1)
    C.SetBranchStatus("cy", 1)
    C.SetBranchStatus("cblktime",1)
    C.SetBranchStatus("cblkatime",1)
    C.SetBranchStatus("cblkid",1)
    C.SetBranchStatus("cblke",1)
    C.SetBranchStatus("tdc",1)
    C.SetBranchStatus("pblkid",1)

    tdc=array.array('d',[0])
    pblkid=array.array('d',[0])
    cx = array.array('d', [0]*10) 
    cy = array.array('d', [0]*10) 
    cblktime=array.array('d',[0]*288)
    cblkatime=array.array('d',[0]*288)
    cblkid=array.array('d',[0]*288)
    cblke=array.array('d',[0]*288)
    
    C.SetBranchAddress("cx", cx)
    C.SetBranchAddress("cy", cy)
    C.SetBranchAddress("cblktime",cblktime)
    C.SetBranchAddress("cblkatime",cblkatime)
    C.SetBranchAddress("cblkid",cblkid)
    C.SetBranchAddress("cblke",cblke)
    C.SetBranchAddress("tdc",tdc)
    C.SetBranchAddress("pblkid",pblkid)
    #_____________________________________
    
    
    #CUT VARS_____________________________________
    C.SetBranchStatus("W2",1)
    C.SetBranchStatus("dx",1)
    C.SetBranchStatus("dy",1)
    
    W2=array.array('d',[0])
    dx=array.array('d',[0])
    dy=array.array('d',[0])
    
    C.SetBranchAddress("W2",W2)
    C.SetBranchAddress("dx",dx)
    C.SetBranchAddress("dy",dy)
    
    #_____________________________________
    
    cx_array=[]
    cy_array=[]
    cblktime_array=[]
    cblkatime_array=[]
    cblkid_array=[]
    cblke_array=[]
    tdc_array=[]
    pblkid_array=[]
    
    
    C.GetEntry(0)
    Entries=C.GetEntries()
    passedcut=0
    for i in range(0,Entries):
        
        C.GetEntry(i)
        
        #cut----------------
        wcut=W2min<W2[0]<W2max
        dxcut=dxmin<dx[0]<dxmax
        dycut=dymin<dy[0]<dymax
        cut=wcut and dxcut and dycut
        #-------------------
        
        if cut:
            passedcut+=1
            cx_array.append(cx)
            cy_array.append(cy)
            tdc_array.append(tdc[0])
            pblkid_array.append(pblkid[0])
            cblktime_array.append(cblktime)
            cblkatime_array.append(cblkatime)

            cblkid_array.append(cblkid)
            cblke_array.append(cblke)
        
    print(f"Events Passing Cut: {passedcut}")
    HCalArrays=[cx_array,cy_array,cblktime_array,cblkid_array,cblke_array,tdc_array,pblkid_array,cblkatime_array]
    return HCalArrays

# To run the function


In [8]:
HCalArrays=Load_HCal("3")
#y=np.array(y)

KeyboardInterrupt: 

Error in <TList::Clear>: A list is accessing an object (0xa844f30) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x9fca130) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa838aa0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa858d40) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa858c30) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa858ad0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa8589e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x8cf6a20) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa818700) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xa7f9bb0) alread

In [ ]:
cblktime=np.array(HCalArrays[2])
atime_array=np.array(HCalArrays[7])
cblkid=np.array(HCalArrays[3]).astype(int)
e_array=np.array(HCalArrays[4])
tdc_array=np.array(HCalArrays[5])
pblkid_array=np.array(HCalArrays[6]).astype(int)

In [ ]:
np.max(atime_array)

In [ ]:

# Initialize 288 histograms
histograms = [[] for _ in range(288)]


In [ ]:
# Fill the histograms
for pblkid, tdc_value in zip(pblkid_array, tdc_array):
    # pblkid ranges from 1 to 288, so subtract 1 to make it 0-indexed
    histograms[pblkid - 1].append(tdc_value)


In [ ]:
def gaussian(x, a, mu, sigma):
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


num_bins = 20

rows, cols = 24, 12

fig, axes = plt.subplots(rows, cols, figsize=(20, 40)) 

mode_array=[]
for i in range(rows):
    for j in range(cols):
        idx = i * cols + j  
        ax = axes[i, j]
        
        counts, bin_edges, _ = ax.hist(histograms[idx], bins=100,range=(-100,200), color='blue', alpha=0.7)
        
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        mode_value = bin_centers[np.argmax(counts)]
        mode_array.append(mode_value)
        
        
        ax.set_title(f"TDC BLK {idx+1}")
        ax.set_ylim(0, 300)
        ax.set_xticks([]) 
        ax.set_yticks([])  

plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, to_rgba
from matplotlib.cm import ScalarMappable, inferno# Assuming `histograms` is the list of filled histograms from earlier

num_bins = 20

rows, cols = 24, 12

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(20, 40), dpi=300)  # Adjust figsize for better visibility

colormap = plt.cm.inferno
normalizer = Normalize(vmin=0, vmax=1000)  # Adjust the max value based on your data range
mode_array = []
for i in range(rows):
    for j in range(cols):
        idx = i * cols + j  # Calculate the index for the histogram
        ax = axes[i, j]
        
        counts, bin_edges, _ = ax.hist(histograms[idx], bins=100, range=(-100, 200), color='black', alpha=1)
        
        total_events = counts.sum()

        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        mode_value = bin_centers[np.argmax(counts)]
        mode_array.append(mode_value)
        bgcolor = to_rgba(colormap(normalizer(total_events)), alpha=0.3)  # Set alpha to 0.5 or any other fixed value

        ax.set_facecolor(bgcolor)
        
        ax.set_title(f"TDC BLK {idx+1}", fontsize=8)
        ax.set_ylim(0, 300)
        ax.set_xticks([])  # Optional: Remove x-ticks for cleaner layout
        ax.set_yticks([])  # Optional: Remove y-ticks for cleaner layout



plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def gaussian(x, amp, mu, sigma):
    return amp * np.exp(-(x - mu)**2 / (2 * sigma**2))

num_bins = 20

rows, cols = 24, 12

fig, axes = plt.subplots(rows, cols, figsize=(20, 40),dpi=300)  # Adjust figsize for better visibility

mean_array = []

for i in range(rows):
    for j in range(cols):
        idx = i * cols + j  # Calculate the index for the histogram
        ax = axes[i, j]
        
        counts, bin_edges, _ = ax.hist(histograms[idx], bins=100, range=(-100, 200), color='blue', alpha=0.7)
        
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        filtered_indices = counts > 10
        filtered_counts = counts[filtered_indices]
        filtered_bin_centers = bin_centers[filtered_indices]
        
        if len(filtered_counts) > 0:
            initial_guess = [max(filtered_counts), filtered_bin_centers[np.argmax(filtered_counts)], np.std(histograms[idx])]
            
            try:
                popt, _ = curve_fit(gaussian, filtered_bin_centers, filtered_counts, p0=initial_guess)
                
                mean_value = popt[1]
                mean_array.append(mean_value)
                
                ax.plot(bin_centers, gaussian(bin_centers, *popt), color='red', lw=2)
                
                
                ax.set_title(f"TDC BLK {idx+1}\nMean: {mean_value:.2f}")
            except:
                mean_array.append(np.nan)
                ax.set_title(f"TDC BLK {idx+1}\nFit Failed")
        else:
            mean_array.append(np.nan)
            ax.set_title(f"TDC BLK {idx+1}\nInsufficient Data")
        ax.set_ylim(0, 300)
        ax.set_xticks([])  # Optional: Remove x-ticks for cleaner layout
        ax.set_yticks([])  # Optional: Remove y-ticks for cleaner layout

plt.tight_layout()
plt.savefig("high_res_histograms.png", dpi=300)

plt.show()


# Idividual

In [ ]:
counts, bin_edges, _ = plt.hist(histograms[114-1], bins=100,range=(-100,200), color='blue', alpha=0.7)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Example Gaussian function
def gaussian(x, amp, mu, sigma):
    return amp * np.exp(-(x - mu)**2 / (2 * sigma**2))

# Assuming you have the counts and bin edges from your histogram
counts, bin_edges, _ = plt.hist(histograms[77-1], bins=100, range=(-100, 200), color='blue', alpha=0.7)

# Calculate bin centers from bin edges
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Filter out bins where counts are less than 40
filtered_indices = counts > 10
filtered_counts = counts[filtered_indices]
filtered_bin_centers = bin_centers[filtered_indices]

# Initial guess for the parameters: [amplitude, mean, standard deviation]
initial_guess = [max(filtered_counts), filtered_bin_centers[np.argmax(filtered_counts)], np.std(histograms[144-1])]

# Fit the Gaussian function to the filtered histogram data
popt, _ = curve_fit(gaussian, filtered_bin_centers, filtered_counts, p0=initial_guess)

# Plot the histogram
plt.hist(histograms[77-1], bins=100, range=(-100, 200), color='blue', alpha=0.7)

# Plot the Gaussian fit (only considering counts > 40)
plt.plot(bin_centers, gaussian(bin_centers, *popt), color='red', lw=2)

# Show the plot
plt.show()

# Print the parameters of the fitted Gaussian
print(f"Amplitude: {popt[0]}")
print(f"Mean (mu): {popt[1]}")
print(f"Standard Deviation (sigma): {popt[2]}")



In [ ]:
np.argmax(counts)

In [ ]:
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2


In [ ]:
bin_edges[24]

In [ ]:
bin_centers[24]

In [ ]:
d=plt.hist(mode_array,bins=50)